In [ ]:
import os
import pandas as pd

from datetime import datetime

## Load COMETKiwi 2022 model

In [ ]:
# log into Huggingface hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)


## Make predictions

In [ ]:
# path to ARC-MTQE directory
main_dir = os.path.dirname(os.getcwd())

In [ ]:
data_dir = os.path.join(main_dir, "data", "mlqe-pe", "data", "catastrophic_errors")

In [ ]:
# save results here
now = datetime.today().strftime("%Y%m%dT%H%M%S")
out_dir = os.path.join(main_dir, "results", "comet_kiwi", now)
os.makedirs(out_dir, exist_ok=True)

In [ ]:
# make predictions for all language pairs listed here
language_pairs = ["encs", "ende", "enja", "enzh"]

In [ ]:
for lp in language_pairs:

    # load data
    path_data = os.path.join(data_dir, f"{lp}_majority_test_blind.tsv")
    df_data = pd.read_csv(path_data, sep='\t', header=None, names=["idx", "source", "target"])
    
    # format for COMETKiwi input: [{"src":"...", "mt":"..."}, {...}] 
    comet_data = []
    for i, row in df_data.iterrows():
        comet_data.append({"src": row["source"], "mt": row["target"]})

    # predict
    model_output = model.predict(comet_data, batch_size=8, gpus=0)

    # save output
    df_results = pd.DataFrame({"idx": df_data["idx"], "comet_score": model_output.scores})
    df_results.to_csv(os.path.join(out_dir, f"{lp}_predictions.csv"), index=False)